<a href="https://colab.research.google.com/github/datapumpernickel/ep_debate/blob/main/translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Oct 30 21:21:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!pip install -U easynmt

In [ ]:
!pip install fasttext
!pip install -U pip transformers
!pip install sentencepiece

# Create EasyNMT instance

Creating an EasyNMT instance and loading a model is easy. You pass the model name you want to use and all needed files are downloaded and cached locally.

In [ ]:
import pandas as pd

full_text = pd.read_csv("https://nextcloud.swp-berlin.org/s/wzcrP4zjSCTyWT2/download")

data = pd.read_csv("https://nextcloud.swp-berlin.org/s/gj7b5xrMNyK8d4A/download")

In [ ]:
data.head()

,Unnamed: 0,text_id,session_id,id_speaker,Sentence_id,sentence
0,0,32,1,1,1,– I declare resumed the session of the Europea...
1,1,6,2,6,1,"– Mr President, Rule 34 of Parliament's Rules ..."
2,2,6,2,6,2,"I was very curious to know why, given that one..."
3,3,6,2,6,3,But now I understand the reason: it is because...
4,4,10,2,4,1,"– Mr President, I think it is a good idea to h..."


In [ ]:
full_text.head()

,text_id,session_id,date_clean,id_speaker,speaker,party,italics,text,url_cre,path_cre,path_toc,orig_orders
0,32,1,1999-01-11,1,President,.,NaN,– I declare resumed the session of the Europea...,https://www.europarl.europa.eu/doceo/document/...,01_raw_data/debates/CRE-4-1999-01-11-ITM-001_E...,01_raw_data/tocs/CRE-4-1999-01-11-TOC_EN.html,1. Resumption of the session
1,6,2,1999-01-11,6,Corbett,(PSE).,(The President of the Commission agreed to thi...,"– Mr President, Rule 34 of Parliament's Rules ...",https://www.europarl.europa.eu/doceo/document/...,01_raw_data/debates/CRE-4-1999-01-11-ITM-002_E...,01_raw_data/tocs/CRE-4-1999-01-11-TOC_EN.html,2. Order of business
2,10,2,1999-01-11,4,Fabre‐Aubrespy,(I‐EDN).,(FR),"– Mr President, I think it is a good idea to h...",https://www.europarl.europa.eu/doceo/document/...,01_raw_data/debates/CRE-4-1999-01-11-ITM-002_E...,01_raw_data/tocs/CRE-4-1999-01-11-TOC_EN.html,2. Order of business
3,13,2,1999-01-11,10,Goepel,(PPE).,(DE),"– Mr President, whilst I believe a vote of con...",https://www.europarl.europa.eu/doceo/document/...,01_raw_data/debates/CRE-4-1999-01-11-ITM-002_E...,01_raw_data/tocs/CRE-4-1999-01-11-TOC_EN.html,2. Order of business
4,14,2,1999-01-11,8,Graefe zu Baringdorf,(V).,(DE),"‐ Mr President, our group requests that the de...",https://www.europarl.europa.eu/doceo/document/...,01_raw_data/debates/CRE-4-1999-01-11-ITM-002_E...,01_raw_data/tocs/CRE-4-1999-01-11-TOC_EN.html,2. Order of business


In [ ]:
full_text.dropna(subset=['text'], inplace=True)
full_text.reset_index(inplace = True)
full_text.head()

,index,text_id,session_id,date_clean,id_speaker,speaker,party,italics,text,url_cre,path_cre,path_toc,orig_orders
0,0,32,1,1999-01-11,1,President,.,NaN,– I declare resumed the session of the Europea...,https://www.europarl.europa.eu/doceo/document/...,01_raw_data/debates/CRE-4-1999-01-11-ITM-001_E...,01_raw_data/tocs/CRE-4-1999-01-11-TOC_EN.html,1. Resumption of the session
1,1,6,2,1999-01-11,6,Corbett,(PSE).,(The President of the Commission agreed to thi...,"– Mr President, Rule 34 of Parliament's Rules ...",https://www.europarl.europa.eu/doceo/document/...,01_raw_data/debates/CRE-4-1999-01-11-ITM-002_E...,01_raw_data/tocs/CRE-4-1999-01-11-TOC_EN.html,2. Order of business
2,2,10,2,1999-01-11,4,Fabre‐Aubrespy,(I‐EDN).,(FR),"– Mr President, I think it is a good idea to h...",https://www.europarl.europa.eu/doceo/document/...,01_raw_data/debates/CRE-4-1999-01-11-ITM-002_E...,01_raw_data/tocs/CRE-4-1999-01-11-TOC_EN.html,2. Order of business
3,3,13,2,1999-01-11,10,Goepel,(PPE).,(DE),"– Mr President, whilst I believe a vote of con...",https://www.europarl.europa.eu/doceo/document/...,01_raw_data/debates/CRE-4-1999-01-11-ITM-002_E...,01_raw_data/tocs/CRE-4-1999-01-11-TOC_EN.html,2. Order of business
4,4,14,2,1999-01-11,8,Graefe zu Baringdorf,(V).,(DE),"‐ Mr President, our group requests that the de...",https://www.europarl.europa.eu/doceo/document/...,01_raw_data/debates/CRE-4-1999-01-11-ITM-002_E...,01_raw_data/tocs/CRE-4-1999-01-11-TOC_EN.html,2. Order of business


In [ ]:
data.dropna(subset=['sentence'], inplace=True)
data.reset_index(inplace = True)

In [ ]:
data

,index,Unnamed: 0,text_id,session_id,id_speaker,Sentence_id,sentence
0,0,0,32,1,1,1,– I declare resumed the session of the Europea...
1,1,1,6,2,6,1,"– Mr President, Rule 34 of Parliament's Rules ..."
2,2,2,6,2,6,2,"I was very curious to know why, given that one..."
3,3,3,6,2,6,3,But now I understand the reason: it is because...
4,4,4,10,2,4,1,"– Mr President, I think it is a good idea to h..."
...,...,...,...,...,...,...,...
2073661,2075220,2075220,179799,14163,1,5,"Das ist aber auch kein Skandal, sondern alles ..."
2073662,2075221,2075221,179800,14164,1,1,Das Protokoll dieser Sitzung wird dem Parlamen...
2073663,2075222,2075222,179800,14164,1,2,"Wenn es keine Einwände gibt, werde ich die in ..."
2073664,2075223,2075223,179801,14165,1,1,Die nächste Tagung findet vom 8. bis zum 9. No...


In [ ]:
#from easynmt import EasyNMT
#model = EasyNMT('m2m_100_418M')

In [ ]:
# download the language model pretrained file
!wget https://dl.fbaipublicfiles.com/nllb/lid/lid218e.bin
!pip install fasttext
import fasttext

pretrained_lang_model = "/content/lid218e.bin" # path of pretrained model file
translation_model = fasttext.load_model(pretrained_lang_model)

--2023-10-30 21:22:28--  https://dl.fbaipublicfiles.com/nllb/lid/lid218e.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.14, 3.163.189.96, 3.163.189.108, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1176355829 (1.1G) [application/octet-stream]
Saving to: ‘lid218e.bin.1’

lid218e.bin.1       100%[===================>]   1.09G   206MB/s    in 7.4s    

2023-10-30 21:22:35 (152 MB/s) - ‘lid218e.bin.1’ saved [1176355829/1176355829]



In [ ]:
#from tqdm import tqdm
#tqdm.pandas()

#data['language'] = data['sentence'].progress_apply(model.language_detection)
#data

In [ ]:
from tqdm import tqdm
tqdm.pandas()
def custom_language_detection(text):
    text = text.replace("\n"," ")
    predictions = translation_model.predict(text, k=1)
    input_lang = predictions[0][0].replace('__label__', '')
    return input_lang

full_text['language'] = full_text['text'].progress_apply(custom_language_detection)
full_text = full_text[["language","text_id","text"]]


100%|██████████| 178378/178378 [05:10<00:00, 573.91it/s]


In [ ]:
full_text.tail(10)

,language,text_id,text
178368,deu_Latn,179936,Procedure :\nMündliche Stimmerklärungen
178369,deu_Latn,179798,Damit ist dieser Tagesordnungspunkt geschlossen.
178370,hrv_Latn,179832,"– Poštovani predsjedavajući, kolegice i kolege..."
178371,eng_Latn,179908,"– , there is growing concern over the ageing o..."
178372,bul_Cyrl,179922,"– Г-н Председател, подкрепих този доклад, защо..."
178373,deu_Latn,179937,Procedure :\nMündliche Stimmerklärungen
178374,deu_Latn,179799,Als nächster Punkt der Tagesordnung folgen die...
178375,deu_Latn,179800,Das Protokoll dieser Sitzung wird dem Parlamen...
178376,deu_Latn,179801,Die nächste Tagung findet vom 8. bis zum 9. No...
178377,deu_Latn,179802,Ich erkläre die Sitzungsperiode des Europäisch...


In [ ]:
data

,index,Unnamed: 0,text_id,session_id,id_speaker,Sentence_id,sentence
0,0,0,32,1,1,1,– I declare resumed the session of the Europea...
1,1,1,6,2,6,1,"– Mr President, Rule 34 of Parliament's Rules ..."
2,2,2,6,2,6,2,"I was very curious to know why, given that one..."
3,3,3,6,2,6,3,But now I understand the reason: it is because...
4,4,4,10,2,4,1,"– Mr President, I think it is a good idea to h..."
...,...,...,...,...,...,...,...
2073661,2075220,2075220,179799,14163,1,5,"Das ist aber auch kein Skandal, sondern alles ..."
2073662,2075221,2075221,179800,14164,1,1,Das Protokoll dieser Sitzung wird dem Parlamen...
2073663,2075222,2075222,179800,14164,1,2,"Wenn es keine Einwände gibt, werde ich die in ..."
2073664,2075223,2075223,179801,14165,1,1,Die nächste Tagung findet vom 8. bis zum 9. No...


In [ ]:


filtered_data = filtered_data[filtered_data['language'] != 'eng_Latn']

# Keep only specific columns
filtered_data = filtered_data[['text_id', 'session_id', 'id_speaker', 'Sentence_id', 'sentence','language']]

filtered_data.reset_index(inplace = True)
filtered_data.drop_duplicates(subset='sentence', inplace = True)
filtered_data

,index,text_id,session_id,id_speaker,Sentence_id,sentence,language
0,41077,3589,178,1,1,\n,cat_Latn
1,41078,3589,178,1,2,On Amendment No 90,cat_Latn
3,56730,4813,247,1,2,On Amendment No 40,fra_Latn
4,58146,4815,255,1,1,IN THE CHAIR: MRS HOFF,yue_Hant
6,168495,13617,762,1,2,(Parliament adopted the legislative resolution...,kor_Hang
...,...,...,...,...,...,...,...
393919,2073658,179799,14163,1,2,Und da darf ich vielleicht auch erläutern:,deu_Latn
393920,2073659,179799,14163,1,3,Die Stimmerklärungen ganz am Schluss werden in...,deu_Latn
393921,2073660,179799,14163,1,4,Und da wir zehn bis 15 Stimmerklärungen haben ...,deu_Latn
393922,2073661,179799,14163,1,5,"Das ist aber auch kein Skandal, sondern alles ...",deu_Latn


In [ ]:
full_text.to_csv("language_detection.csv")
filtered_data = pd.merge(data, full_text, on='text_id', how='left').to_csv("language_detection_sentence.csv")

KeyboardInterrupt: ignored

# Sentence Translation
When you have individual sentences to translate, you can call the method `translate_sentences`.

In [ ]:
#translations = model.translate(filtered_data.sentence, target_lang='en')

In [ ]:
filtered_data['language'].unique()

array(['cat_Latn', 'fra_Latn', 'yue_Hant', 'kor_Hang', 'glg_Latn',
       'spa_Latn', 'deu_Latn', 'arb_Arab', 'bod_Tibt', 'krc_Cyrl',
       'zho_Hans', 'ita_Latn', 'lit_Latn', 'por_Latn', 'swe_Latn',
       'pol_Latn', 'ell_Grek', 'slk_Latn', 'hun_Latn', 'ces_Latn',
       'dan_Latn', 'mlt_Latn', 'nld_Latn', 'lvs_Latn', 'fin_Latn',
       'est_Latn', 'slv_Latn', 'gle_Latn', 'ron_Latn', 'bul_Cyrl',
       'oci_Latn', 'ind_Latn', 'ast_Latn', 'vie_Latn', 'ilo_Latn',
       'hrv_Latn', 'kab_Latn'], dtype=object)

In [ ]:

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
checkpoint = 'facebook/nllb-200-distilled-600M'
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from tqdm.notebook import tqdm
import pandas as pd
from transformers import pipeline

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
# Initialize an empty list to store the translated sentences
all_translated_sentences = []

# Loop through each unique language in the 'language' column with a progress bar
for current_language in tqdm(filtered_data['language'].unique(), desc="Translating by Language"):
    # Filter the DataFrame to only include rows with the current language
    language_specific_data = filtered_data[filtered_data['language'] == current_language]

    # Initialize the translation pipeline for the current source language
    translation_pipeline = pipeline('translation',
                                    model=model,
                                    tokenizer=tokenizer,
                                    device = 0,
                                    src_lang=current_language,
                                    tgt_lang='eng_Latn',
                                    max_length=500)

    # Process each sentence in the language-specific DataFrame with a progress bar
    for sentence in tqdm(language_specific_data.sentence, desc=f"Translating Sentences in {current_language}", leave=False):
        translation_output = translation_pipeline(sentence)
        all_translated_sentences.append(translation_output[0]['translation_text'])

# Create a new DataFrame to store the translated sentences
translated_dataframe = pd.DataFrame({
    'translated_sentence': all_translated_sentences
})




Translating by Language:   0%|          | 0/37 [00:00<?, ?it/s]

Translating Sentences in cat_Latn:   0%|          | 0/3 [00:00<?, ?it/s]

Translating Sentences in fra_Latn:   0%|          | 0/59075 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
import json
from transformers import pipeline

# Initialize an empty list to store the translated sentences
all_translated_sentences = []

# Open the JSONL file to store the translations
with open('translated_sentences.jsonl', 'a+') as f:
    # Loop through each unique language in the 'language' column with a progress bar
    for current_language in tqdm(filtered_data['language'].unique(), desc="Translating by Language"):
        # Filter the DataFrame to only include rows with the current language
        language_specific_data = filtered_data[filtered_data['language'] == current_language]

        # Initialize the translation pipeline for the current source language
        translation_pipeline = pipeline('translation',
                                        model=model,
                                        tokenizer=tokenizer,
                                        device=0,  # Set to 0 to run on the first available GPU
                                        src_lang=current_language,
                                        tgt_lang='eng_Latn',
                                        max_length=500)

        # Process each sentence in the language-specific DataFrame with a progress bar
        for i, row in tqdm(language_specific_data.iterrows(), desc=f"Translating Sentences in {current_language}", leave=False):
            text_id = row['text_id']
            sentence_id = row['Sentence_id']
            sentence = row['sentence']

            # Check if this text_id and Sentence_id already exist in the JSONL file
            f.seek(0)
            if any(json.loads(line).get('text_id') == text_id and json.loads(line).get('Sentence_id') == sentence_id for line in f):
                continue

            # Translate the sentence
            translation_output = translation_pipeline(sentence)
            translated_text = translation_output[0]['translation_text']

            # Write the translation, text_id, and Sentence_id to the JSONL file
            json.dump({"text_id": text_id, "Sentence_id": sentence_id, "translated_sentence": translated_text}, f)
            f.write('\n')


Translating by Language:   0%|          | 0/37 [00:00<?, ?it/s]

Translating Sentences in cat_Latn: 0it [00:00, ?it/s]

Translating Sentences in fra_Latn: 0it [00:00, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
from transformers import pipeline

# Initialize an empty list to store the translated sentences
all_translated_sentences = []

# Batch size for translation
batch_size = 168

# Loop through each unique language in the 'language' column with a progress bar
for current_language in tqdm(filtered_data['language'].unique(), desc="Translating by Language"):
    # Filter the DataFrame to only include rows with the current language
    language_specific_data = filtered_data[filtered_data['language'] == current_language]

    # Initialize the translation pipeline for the current source language
    translation_pipeline = pipeline('translation',
                                    model=model,
                                    tokenizer=tokenizer,
                                    device = 0,
                                    src_lang=current_language,
                                    tgt_lang='eng_Latn',
                                    max_length=500)

    # Process each sentence in the language-specific DataFrame in batches
    for i in tqdm(range(0, len(language_specific_data), batch_size),
                  desc=f"Translating Sentences in {current_language}",
                  leave=False):
        batch = language_specific_data.sentence.iloc[i:i + batch_size].tolist()
        translation_outputs = translation_pipeline(batch)
        translated_batch = [t['translation_text'] for t in translation_outputs]
        all_translated_sentences.extend(translated_batch)

# Create a new DataFrame to store the translated sentences
translated_dataframe = pd.DataFrame({
    'translated_sentence': all_translated_sentences
})


Translating by Language:   0%|          | 0/37 [00:00<?, ?it/s]

Translating Sentences in cat_Latn:   0%|          | 0/1 [00:00<?, ?it/s]

Translating Sentences in fra_Latn:   0%|          | 0/352 [00:00<?, ?it/s]

KeyboardInterrupt: ignored